# **Group 5 ED(C) Deep Learning Approach with the use of Transformers - Demo on the test set**

Import necessary libraries

In [2]:
from claims_dataset_transformer import ClaimEvidenceDataset
from evidence_classifier_transformer import EvidenceClassifier
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader

Load the test set

In [3]:
test_df = pd.read_csv('data/test.csv')
test_df.head()

,Claim,Evidence
0,We should further exploit geothermal energy,Taxpayer funding of research and development o...
1,We should prohibit corporal punishment,"Regarding discipline, Sukhmani writes that cor..."
2,We should ban male infant circumcision,"Benatar and Benatar (2003) argue that ""it is f..."
3,We should ban trans fats usage in food,"Each KIND bar is gluten free, dairy free, non ..."
4,We should ban boxing,About Feng Keshan and Meihuaquan: In the 1800s...


Load the model and output its structure

The 'ED_model_C.pth' model can be downloaded from the following link: https://drive.google.com/drive/folders/1TWv5UKNsNeQGxafx3GQf87Dc8vcp5V8t?usp=sharing

In [4]:
model_path = 'ED_model_C.pth'
model = EvidenceClassifier()
model.load_state_dict(torch.load(model_path))
print(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

EvidenceClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

Define the predict function

In [9]:
def predict(test_csv, model_path):
  # Set device to cuda to run on GPU
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  # Load the model state dictionary
  model = EvidenceClassifier().to(device)
  model.load_state_dict(torch.load(model_path, map_location=device))

  # load the data to evaluate on (in this case the dev dataset)
  test_dataset = ClaimEvidenceDataset(test_csv,is_test=True)
  test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
  # set the model to evaluation mode
  model.eval()

  all_predictions = []
  all_probabilities = []

  with torch.no_grad():
    for batch in test_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      token_type_ids = batch['token_type_ids'].to(device)

      logits = model(input_ids, attention_mask, token_type_ids)
      probabilities = torch.nn.functional.softmax(logits, dim=1)
      predictions = torch.argmax(probabilities, dim=1)

      all_predictions.extend(predictions.cpu().numpy())
      all_probabilities.extend(probabilities.cpu().numpy())


  df = pd.read_csv(test_csv)
  df['prediction'] = all_predictions
  return df

Run the model in inference mode on the test set and print first few predictions

In [10]:
predictions = predict('data/test.csv', 'ED_model_C.pth')

for i in range(10):
  print(f"Evidence: {predictions['Evidence'][i]}")
  print(f"Claim: {predictions['Claim'][i]}")
  print(f"Prediction: {predictions['prediction'][i]}")

Evidence: Taxpayer funding of research and development of solar energy, geothermal energy, and marine and hydrokinetic renewable energy technologies.
Claim: We should further exploit geothermal energy
Prediction: 0
Evidence: Regarding discipline, Sukhmani writes that corporal punishment is taboo at the school [REF].
Claim: We should prohibit corporal punishment
Prediction: 1
Evidence: Benatar and Benatar (2003) argue that "it is far from obvious that circumcision reduces sexual pleasure," and that "it is far from clear that non-circumcision leaves open a future person's options in every regard."
Claim: We should ban male infant circumcision
Prediction: 0
Evidence: Each KIND bar is gluten free, dairy free, non GMO, trans fat free, low GI and low in sodium.
Claim: We should ban trans fats usage in food
Prediction: 0
Evidence: About Feng Keshan and Meihuaquan: In the 1800s, Feng Keshan developed the Mei Hua Quan or Plum Flower Boxing style.
Claim: We should ban boxing
Prediction: 0
Eviden

Save the predictions

In [14]:
predictions['prediction'].to_csv('predictions/Group_5_C.csv', index=False)

Run the model on User Inputs

In [18]:
claim = input("Enter a claim: ")
evidence = input("Enter an evidence: ")

df = pd.DataFrame({'Claim': [claim], 'Evidence': [evidence]})
df.to_csv('data/input.csv', index=False)
predictions = predict('data/input.csv', 'ED_model_C.pth')
prediction = "relevant" if predictions['prediction'][0] == 1 else "irrelevant"
print(f"Prediction: {prediction}")

Enter a claim: We should ban fast food
Enter an evidence: "The fast food industry is a popular target for critics, from anti-globalization activists like José Bové to vegetarian activist groups such as PETA "
Prediction: irrelevant
